In [1]:
import tensorflow
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [2]:
import os
print(os.listdir("../input"))

['pokemon1', 'pokemon']


In [3]:
X_train = np.load("../input/pokemon/X_train.npy")
Y_train = np.load("../input/pokemon/Y_train.npy")
x_test = np.load("../input/pokemon/X_test.npy")

In [4]:
Y_train = np_utils.to_categorical(Y_train)

In [5]:
print ((X_train.shape, Y_train.shape))

((1252, 227, 227, 3), (1252, 10))


In [6]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

In [7]:
# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

In [8]:
# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [9]:
adam = Adam(lr = 0.00003)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [10]:
from keras.callbacks import ModelCheckpoint

In [11]:
checkpoint = ModelCheckpoint("model.h5", save_best_only=True,monitor='val_acc')
hist = model.fit(X_train, Y_train, epochs = 100, validation_split = 0.2, shuffle = True, batch_size = 64,callbacks = [checkpoint])


Train on 1001 samples, validate on 251 samples
Epoch 1/100
1001/1001 [==============================] - 6s 6ms/step - loss: 2.7583 - acc: 0.1648 - val_loss: 1.6968 - val_acc: 0.4462
Epoch 2/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.0883 - acc: 0.3077 - val_loss: 1.4735 - val_acc: 0.4940
Epoch 3/100
1001/1001 [==============================] - 2s 2ms/step - loss: 1.7101 - acc: 0.4186 - val_loss: 1.2225 - val_acc: 0.5976
Epoch 4/100
1001/1001 [==============================] - 2s 2ms/step - loss: 1.4348 - acc: 0.5075 - val_loss: 1.1073 - val_acc: 0.6614
Epoch 5/100
1001/1001 [==============================] - 2s 2ms/step - loss: 1.2514 - acc: 0.5904 - val_loss: 1.0396 - val_acc: 0.6733
Epoch 6/100
1001/1001 [==============================] - 2s 2ms/step - loss: 0.9694 - acc: 0.6753 - val_loss: 0.9516 - val_acc: 0.7211
Epoch 7/100
1001/1001 [==============================] - 2s 2ms/step - loss: 0.9384 - acc: 0.6853 - val_loss: 0.9321 - val_acc: 0.7171
Epoch 8/

In [12]:
from keras.models import load_model
r_model = load_model("model.h5")

In [13]:
y_predict = r_model.predict_classes(x_test)


In [14]:
print(y_predict.shape)

(313,)


In [15]:
y_predict

array([6, 0, 9, 9, 0, 7, 5, 7, 1, 6, 7, 3, 8, 5, 9, 4, 1, 4, 6, 2, 0, 8,
       3, 5, 5, 2, 9, 8, 6, 1, 2, 6, 7, 9, 1, 4, 5, 6, 7, 7, 7, 4, 3, 6,
       7, 5, 1, 3, 4, 6, 2, 7, 9, 2, 2, 4, 6, 0, 4, 7, 2, 5, 6, 9, 1, 2,
       1, 9, 0, 9, 1, 2, 2, 2, 5, 4, 2, 6, 6, 6, 9, 5, 1, 6, 5, 6, 2, 1,
       8, 6, 9, 4, 2, 5, 2, 8, 5, 7, 9, 5, 1, 6, 1, 7, 8, 7, 5, 6, 6, 8,
       4, 4, 5, 8, 1, 9, 1, 6, 9, 1, 3, 4, 2, 1, 5, 6, 2, 7, 7, 1, 3, 1,
       2, 2, 8, 2, 2, 1, 8, 5, 5, 2, 9, 7, 0, 0, 9, 2, 5, 0, 5, 7, 3, 3,
       3, 7, 7, 6, 7, 6, 9, 5, 7, 1, 0, 7, 1, 0, 1, 6, 6, 2, 9, 7, 7, 7,
       8, 8, 7, 2, 9, 1, 5, 9, 9, 7, 4, 9, 9, 0, 6, 7, 7, 9, 6, 9, 8, 7,
       2, 0, 6, 6, 4, 7, 5, 8, 2, 9, 7, 2, 6, 6, 6, 6, 6, 6, 3, 5, 5, 7,
       6, 0, 6, 5, 8, 7, 7, 4, 6, 8, 9, 5, 7, 2, 5, 9, 7, 2, 5, 8, 8, 2,
       6, 5, 5, 4, 9, 4, 4, 5, 2, 8, 8, 8, 7, 6, 2, 7, 6, 8, 9, 0, 8, 6,
       2, 9, 3, 7, 5, 2, 2, 9, 8, 5, 5, 9, 9, 7, 3, 2, 3, 3, 1, 6, 1, 5,
       4, 2, 6, 9, 1, 6, 8, 5, 8, 6, 3, 4, 2, 0, 0,

In [16]:
y_test = np.load("../input/pokemon1/Y_test.npy")

In [17]:
print(y_test.shape)

(313,)


In [18]:
y_test

array([6, 8, 2, 9, 0, 7, 5, 7, 1, 8, 7, 3, 8, 5, 2, 4, 1, 4, 6, 2, 5, 8,
       3, 5, 5, 2, 4, 4, 4, 1, 2, 6, 4, 1, 1, 4, 5, 4, 7, 7, 7, 4, 3, 6,
       2, 5, 1, 5, 4, 6, 2, 7, 2, 2, 7, 4, 6, 0, 4, 7, 9, 5, 2, 9, 1, 2,
       1, 1, 5, 9, 1, 2, 2, 2, 5, 5, 2, 6, 6, 6, 9, 5, 1, 6, 5, 6, 4, 1,
       8, 6, 9, 4, 2, 5, 2, 2, 0, 7, 5, 5, 9, 6, 1, 7, 3, 7, 5, 6, 8, 4,
       4, 0, 5, 8, 1, 5, 1, 6, 9, 1, 3, 4, 2, 1, 5, 2, 2, 6, 7, 9, 3, 1,
       2, 2, 3, 2, 2, 6, 8, 5, 5, 2, 9, 7, 0, 0, 1, 2, 3, 3, 5, 7, 3, 3,
       3, 9, 7, 6, 7, 6, 3, 5, 7, 2, 5, 7, 1, 0, 1, 6, 6, 2, 2, 7, 7, 7,
       8, 8, 4, 2, 1, 1, 5, 1, 9, 7, 4, 6, 9, 0, 6, 7, 2, 4, 6, 9, 8, 7,
       2, 0, 6, 6, 4, 6, 5, 8, 2, 9, 6, 2, 8, 6, 6, 6, 2, 6, 3, 8, 5, 6,
       6, 0, 2, 1, 4, 7, 7, 4, 6, 8, 9, 2, 7, 2, 5, 1, 7, 2, 5, 8, 2, 4,
       6, 3, 5, 4, 9, 4, 4, 5, 2, 8, 8, 8, 7, 6, 2, 7, 6, 8, 9, 0, 8, 6,
       2, 9, 3, 7, 5, 2, 2, 9, 8, 5, 5, 9, 3, 7, 3, 2, 3, 3, 1, 4, 1, 5,
       4, 2, 6, 9, 9, 6, 4, 5, 8, 6, 3, 4, 2, 6, 0,

In [19]:
accuracy = np.mean(y_predict == y_test)
accuracy


0.7795527156549521

In [20]:
print(y_predict.dtype)

np.savetxt('f1.csv',y_predict)

int64
